In [101]:
import ortools
from ortools.init.python import init
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# 0. Data import and solver set up

In [102]:
distances = pd.read_excel('distances.xlsx', index_col=0, skiprows=1, usecols=range(1, 6))
print("Shape of distances dataset : ",distances.shape)
display(distances.head())
index_values = pd.read_excel('indexValues.xlsx', header=None, names=["indexValue"], usecols=[1])
print("Shape of indexValues dataset : ",index_values.shape)
display(index_values.head())
x_origin = pd.read_excel('x_origin.xlsx', index_col=0,skiprows=1, usecols=range(1, 6))
print("Shape of x' dataset : ",x_origin.shape)
display(x_origin.head())

Shape of distances dataset :  (22, 4)


,1,2,3,4
1,16.16,24.08,24.32,21.12
2,19.00,26.47,27.24,17.33
3,25.29,32.49,33.42,12.25
4,0.00,7.93,8.31,36.12
5,3.07,6.44,7.56,37.37


Shape of indexValues dataset :  (22, 1)


,indexValue
0,0.1609
1,0.1164
2,0.1026
3,0.1516
4,0.0939


Shape of x' dataset :  (22, 4)


,1,2,3,4
x'ij,,,,
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,1,0,0,0
5,1,0,0,0


# 1. Variables

In [103]:
def set_variables(index_values,distances):
    num_bricks = len(index_values)
    num_sr = len(distances.columns)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    # assignment[i][j] = 1 if brick i is assigned to SR j
    assignment = [[solver.BoolVar(f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, assignment, num_bricks, num_sr

# 2. Constraints

In [104]:
def constraint_assignment_bricks(solver, assignment, num_bricks, num_sr, nb_sr_to_assign):
    for i in range(num_bricks):
        solver.Add(sum(assignment[i][j] for j in range(num_sr)) >= 1)
        solver.Add(sum(assignment[i][j] for j in range(num_sr)) <= nb_sr_to_assign)

def constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval):
    for j in range(num_sr):
        total_weight = solver.Sum(assignment[i][j] * index_values.iloc[i,0] for i in range(num_bricks))
        solver.Add(total_weight >= min(condition_interval))
        solver.Add(total_weight <= max(condition_interval))

def apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval, nb_sr_to_assign):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr,  nb_sr_to_assign)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

# 3. Objective functions

In [105]:
# Objective function: minimize weighted distance + weight-based disruption penalty
def get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant):
    objective = solver.Objective()
    for i in range(num_bricks):
        for j in range(num_sr):
            # Weighted distance cost
            distance_cost = distances.iloc[i,j]
            
            # Disruption penalty if reassigned to a different SR
            if (i + 1) in x_origin and x_origin.iloc[i + 1,0] != j + 1:
                disruption_penalty = penalty_constant * index_values.iloc[i,0]
            else:
                disruption_penalty = 0

            # Total cost for assigning brick i to SR j
            total_cost = distance_cost + disruption_penalty
            objective.SetCoefficient(assignment[i][j], total_cost)

    objective.SetMinimization()

    return objective

# 4. Solve

In [106]:
def get_stats(assignment, num_sr, num_bricks, objective):
    sr_weights = [0.0] * num_sr
    sr_distances = [0.0] * num_sr
    for j in range(num_sr):
        sr_bricks = []
        for i in range(num_bricks):
            if assignment[i][j].solution_value() > 0.5:
                sr_weights[j] += index_values.iloc[i,0]
                sr_distances[j] += distances.iloc[i,j]
                sr_bricks.append(i)
        print(f"SR {j+1} total weight is {round(sr_weights[j],2)} / total distances is {round(sr_distances[j],2)} / assigned bricks are {sr_bricks}")
    # Calculate the sum of (sum of weights per SR * sum of distances per SR)
    total_distances = sum(sr_distances)
    total_weighted_distance = sum(sr_weights[j] * sr_distances[j] for j in range(num_sr))
    print('Sum of distances:', total_distances)
    print('Sum of weighted distances:', total_weighted_distance)
    print('Optimal Objective Value:', objective.Value())
    
def solve_print(solver, assignment, objective, num_bricks, num_sr):
    # Solve the problem
    status = solver.Solve()
    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        get_stats(assignment, num_sr, num_bricks, objective)
        
    else:
        print('No optimal solution found.')


# 5. Get Results

In [107]:
def get_results(distances,index_values, nb_sr_to_assign, condition_interval, penalty_constant):
    solver, assignment, num_bricks, num_sr = set_variables(index_values,distances)
    apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval, nb_sr_to_assign)
    objective = get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant)
    solve_print(solver, assignment, objective, num_bricks, num_sr)

In [108]:
condition_interval = [0.8,1.2]
penalty_constant = 10
nb_sr_to_assign = 1
get_results(distances, index_values, nb_sr_to_assign, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.04 / total distances is 64.37 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19]
SR 2 total weight is 1.04 / total distances is 7.53 / assigned bricks are [10, 12, 13, 17]
SR 3 total weight is 1.11 / total distances is 6.57 / assigned bricks are [9, 14, 15, 16]
SR 4 total weight is 0.8 / total distances is 76.13 / assigned bricks are [0, 1, 2, 20, 21]
Sum of distances: 154.6
Sum of weighted distances: 143.09896
Optimal Objective Value: 158.39900000000003


# 6. Questions

### 6.1. Do the obtained solutions have properties that are worth bringing to the attention of the decision maker?

- **Total Weighted Distance**: The solution has a total weighted distance of approximately 143.1. This metric helps the decision maker understand the overall efficiency and cost of the current SR (Sales Representatives) allocation.

- **Weight Balance**: Each SR has a total weight within the constraint interval (between 0.8 and 1.2), indicating a balanced workload. SRs 1, 2, and 3 are near the upper limit, while SR 4 is close to the minimum requirement.

- **Distance Distribution**: SRs 1 and 4 have significantly higher total distances (64.37 and 76.13 respectively) compared to SRs 2 and 3 (7.53 and 6.57). This suggests that SRs 1 and 4 cover more geographically dispersed bricks, which may imply higher travel times or costs.

- **Assignment of Bricks**: SRs are assigned a varied number of bricks, with SR 1 covering nine bricks while SR 2, 3, and 4 cover fewer. This indicates that some SRs may face higher workload concentration in certain areas, which could impact operational efficiency.

- **Assignment distruption**: good metric to evaluate how much changes we have between each assignment

### 6.2. Discuss the properties of the current solution in comparison to other solutions obtained.

- **Workload Distribution**: Compared to other solutions where the workload might not be as evenly distributed, this solution appears to have achieved a balance across SRs, ensuring that no SR is overwhelmed by an excessive workload.

- **Distance Optimization**: By minimizing the weighted distance, this solution achieves a relatively low-cost allocation. However, if there are alternate solutions with slightly higher costs but more balanced distances across SRs, these could be considered for improved efficiency.

- **Disruption Penalty**: This solution minimizes reassignment of bricks compared to the origin (x_origin) due to the disruption penalty applied. This minimizes costs associated with changing SR responsibilities.

### 6.3. Varying workload is currently an objective taken into account in the form of constraints. We can get different sets of effective solutions and tradeoffs with other goals by tightening and relaxing this constraint. Try at least one interval and discuss the result (you can use [0.9, 1.1])

In [109]:
condition_interval = [0.9,1.1]
penalty_constant = 10
nb_sr_to_assign = 1
get_results(distances, index_values, nb_sr_to_assign, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 0.94 / total distances is 46.88 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18]
SR 2 total weight is 1.07 / total distances is 9.22 / assigned bricks are [10, 12, 13, 16]
SR 3 total weight is 1.09 / total distances is 6.99 / assigned bricks are [9, 14, 15, 17]
SR 4 total weight is 0.9 / total distances is 99.34 / assigned bricks are [0, 1, 2, 19, 20, 21]
Sum of distances: 162.43
Sum of weighted distances: 151.111496
Optimal Objective Value: 166.22899999999996


SRs that were previously assigned workloads close to the upper and lower limits (like SR 4) will likely be forced to take on additional bricks or reduce their assigned bricks, shifting assignments more uniformly across SRs.
This tighter constraint increases the total weighted distance since the model has fewer assignment options to minimize cost. If the workload becomes more balanced, some SRs may cover fewer distances overall, while others may need to take on more, resulting in a trade-off between cost and balanced distribution.

### 6.4. How to model the case for partially assigning bricks (i.e. assign a brick to multiple SR)? Implement this and compare the results.

In [110]:
for i in range(1,len(distances.columns)+1):
    condition_interval = [0.8,1.2]
    penalty_constant = 10
    nb_sr_to_assign = i
    print(f"NUMBER OF MAX ASSIGNMENT POSSIBLE PER BRICK: {i}")
    get_results(distances, index_values, nb_sr_to_assign, condition_interval,penalty_constant)
    print('-'*100)

NUMBER OF MAX ASSIGNMENT POSSIBLE PER BRICK: 1
Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.04 / total distances is 64.37 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19]
SR 2 total weight is 1.04 / total distances is 7.53 / assigned bricks are [10, 12, 13, 17]
SR 3 total weight is 1.11 / total distances is 6.57 / assigned bricks are [9, 14, 15, 16]
SR 4 total weight is 0.8 / total distances is 76.13 / assigned bricks are [0, 1, 2, 20, 21]
Sum of distances: 154.6
Sum of weighted distances: 143.09896
Optimal Objective Value: 158.39900000000003
----------------------------------------------------------------------------------------------------
NUMBER OF MAX ASSIGNMENT POSSIBLE PER BRICK: 2
Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.04 / total distances is 64.37 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19]
SR 2 total weight is 1.04 / total distances is 7.53 / assigned bricks are [10, 12, 13, 17]
SR 3 total w

### 6.5.  If the demand increases uniformly in all bricks (for example + 20%, it may be necessary to hire a new sales representative. There is the question of where to locate his office (center brig)

PS: To optimize the SR’s location, calculate the geographical center (average coordinates) of the bricks assigned to existing SRs with the heaviest demand.
Potential Adjustments: Redistribute bricks so that the additional SR takes on part of the workload, aiming to balance distances and demand while minimizing total weighted distance.

### 6.6.  The location of the "center bricks" (SR offices) has a significant impact on the distance traveled by the SRs. An important question is to generalize the model so as to allow a modification of the "center bricks"; this requires considering additional binary variables. (It should be noted that some of the bricks are not good candidates as a "center brig", which makes it possible to reduce the number of variables).

To generalize the model by allowing SR office relocations, we have to:

- **Add Binary Variables**: Introduce binary variables to indicate the selection of certain bricks as new SR centers. Not all bricks are candidates, so restrict the variable to a subset of bricks that are potential centers.

- **Objective Function Adjustment**: Incorporate the distances from each brick to the selected SR centers, modifying the objective to minimize both weighted distance and potential disruption penalties due to reassignment. With this the model can adapt dynamically to changes in demand and geography by choosing optimal SR office locations, potentially lowering travel distances and improving response times.

Each modification builds on the initial model to address different logistical scenarios, improving operational efficiency under changing conditions.

### 6.7  How to incorporate in the model the SRs' preferences for their area?

To incorporate SRs' area preferences, we could add:

- **Preference Penalty in Objective Function**: Add a penalty to the objective for assignments that do not match SRs’ preferences, encouraging alignment without imposing hard constraints.

- **Weighted Assignments**: Assign weights to bricks based on SRs' preferences, prioritizing preferred areas.